In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import pickle

In [2]:
PREDICT_FORWARD = 261

In [3]:
df = pd.read_csv("daily_returns4.csv")
df = df.set_index(['TICKER', "date"])
df.sort_index(inplace = True)

revtq = df["revtq"]
df = df.drop(columns = ["revtq"])
revtq = revtq.apply(lambda x: np.log(x + np.exp(1)))
df["modrevtq"] = revtq

In [4]:
df

PERMNO    BIDLO   ASKHI    PRC        VOL    BID    ASK  \
TICKER date                                                             
ADBE   40    75510.0  39.2900  39.935  39.31  3671897.0  39.30  39.31   
       41    75510.0  38.9600  39.880  39.83  2398125.0  39.83  39.84   
       42    75510.0  39.4500  40.470  40.46  2441467.0  40.46  40.47   
       43    75510.0  40.6800  41.490  41.46  4309693.0  41.46  41.47   
       44    75510.0  41.1300  41.770  41.29  2878263.0  41.28  41.29   
...              ...      ...     ...    ...        ...    ...    ...   
ZUO    1506  17382.0  15.5600  16.850  16.36   550036.0  16.37  16.39   
       1507  17382.0  16.3700  17.820  17.81  1092333.0  17.80  17.81   
       1508  17382.0  16.6200  17.920  17.90  1011304.0  17.90  17.91   
       1509  17382.0  17.4100  18.160  17.65  1141001.0  17.65  17.66   
       1510  17382.0  17.7601  18.150  18.14  1125526.0  18.13  18.14   

             OPENPRC   NUMTRD      RETX  ...     RET52     RET53     RET54  \
TICKER date                              ...                                 
ADBE   40      39.54  23806.0 -0.007323  ...  0.000000  0.000000  0.000000   
       41      39.30  17542.0  0.013228  ...  0.000000  0.000000  0.000000   
       42      39.74  16178.0  0.015817  ...  0.000000  0.000000  0.000000   
       43      40.76  27325.0  0.024716  ...  0.000000  0.000000  0.000000   
       44      41.47  20295.0 -0.004100  ...  0.000000  0.000000  0.000000   
...              ...      ...       ...  ...       ...       ...       ...   
ZUO    1506    15.67      NaN  0.017413  ... -0.014544 -0.057656 -0.007505   
       1507    16.68      NaN  0.088631  ... -0.058015 -0.014544 -0.057656   
       1508    17.31      NaN  0.005053  ... -0.036737 -0.058015 -0.014544   
       1509    17.94      NaN -0.013966  ...  0.015143 -0.036737 -0.058015   
       1510    17.95      NaN  0.027762  ...  0.093370  0.015143 -0.036737   

                RET55     RET56     RET57     RET58     RET59     RET60  \
TICKER date                                                               
ADBE   40    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
       41    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
       42    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
       43    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
       44    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
...               ...       ...       ...       ...       ...       ...   
ZUO    1506  0.010427 -0.000474 -0.016768 -0.070965  0.009611  0.020508   
       1507 -0.007505  0.010427 -0.000474 -0.016768 -0.070965  0.009611   
       1508 -0.057656 -0.007505  0.010427 -0.000474 -0.016768 -0.070965   
       1509 -0.014544 -0.057656 -0.007505  0.010427 -0.000474 -0.016768   
       1510 -0.058015 -0.014544 -0.057656 -0.007505  0.010427 -0.000474   

             modrevtq  
TICKER date            
ADBE   40    6.918291  
       41    6.918291  
       42    6.918291  
       43    6.918291  
       44    6.918291  
...               ...  
ZUO    1506  4.160043  
       1507  4.160043  
       1508  4.160043  
       1509  4.160043  
       1510  4.160043  

[72563 rows x 97 columns]

In [5]:
df.columns

Index(['PERMNO', 'BIDLO', 'ASKHI', 'PRC', 'VOL', 'BID', 'ASK', 'OPENPRC',
       'NUMTRD', 'RETX', 'gvkey', 'fyearq', 'actq', 'ancq', 'aoq', 'atq',
       'aul3q', 'ceqq', 'cheq', 'chq', 'cogsq', 'cshiq', 'cshoq', 'dpq',
       'drcq', 'drltq', 'lctq', 'ltq', 'rdipq', 'stkcpay', 'teqq', 'tfvaq',
       'tfvlq', 'xrdq', 'xsgaq', 'RET0', 'RET1', 'RET2', 'RET3', 'RET4',
       'RET5', 'RET6', 'RET7', 'RET8', 'RET9', 'RET10', 'RET11', 'RET12',
       'RET13', 'RET14', 'RET15', 'RET16', 'RET17', 'RET18', 'RET19', 'RET20',
       'RET21', 'RET22', 'RET23', 'RET24', 'RET25', 'RET26', 'RET27', 'RET28',
       'RET29', 'RET30', 'RET31', 'RET32', 'RET33', 'RET34', 'RET35', 'RET36',
       'RET37', 'RET38', 'RET39', 'RET40', 'RET41', 'RET42', 'RET43', 'RET44',
       'RET45', 'RET46', 'RET47', 'RET48', 'RET49', 'RET50', 'RET51', 'RET52',
       'RET53', 'RET54', 'RET55', 'RET56', 'RET57', 'RET58', 'RET59', 'RET60',
       'modrevtq'],
      dtype='object')

In [6]:
s = set([])
for row in df.iterrows():
    idx = row[0]
    date = idx[1]
    
    if (idx[0], idx[1] + PREDICT_FORWARD) in df.index:
        s.add(idx[0])
        
print(len(s))  

67


In [7]:
s = list(s)
np.random.shuffle(s)
train = s[:4 * len(s) // 5]
test = s[4 * len(s) // 5:]
s = set(s)
print(train)
print(test)

['JCOM', 'AMZN', 'APPF', 'SHOP', 'NTNX', 'EGHT', 'CHKP', 'CRM', 'BKI', 'MTCH', 'ETSY', 'SPLK', 'HUBS', 'MDB', 'BOX', 'SQ', 'SAIL', 'PYPL', 'OKTA', 'EXPE', 'WDAY', 'FEYE', 'SWI', 'VEEV', 'GWRE', 'RNG', 'INST', 'WIX', 'NEWR', 'TEAM', 'RPD', 'AYX', 'TDOC', 'FIVN', 'COUP', 'TTD', 'CLDR', 'TLND', 'LOGM', 'NFLX', 'APPN', 'WK', 'BAND', 'PAYC', 'TWLO', 'BL', 'PANW', 'INTU', 'SSTK', 'QLYS', 'EVBG', 'YEXT', 'TWOU']
['TRIP', 'BLKB', 'FB', 'MIME', 'PCTY', 'NOW', 'ADSK', 'ZEN', 'PFPT', 'ADBE', 'RP', 'QTWO', 'EBAY', 'GDDY']


In [8]:
X = [[], []]
Y = [[], []]

for row in tqdm(df.iterrows()):
    idx = row[0]
    
    train_test = -1
    if idx[0] in train:
        train_test = 0
    elif idx[0] in test:
        train_test = 1
    else:
        continue
        
    nxtidx = (idx[0], idx[1] + PREDICT_FORWARD)
    if nxtidx in df.index:
        Y[train_test].append(df.at[nxtidx, "modrevtq"] - row[1]["modrevtq"])
        x = np.asarray(row[1], dtype='float64')
        x = np.nan_to_num(x)
        X[train_test].append(x)
        
for i in range(2):
    X[i] = np.asarray(X[i])
    Y[i] = np.asarray(Y[i])
    Y[i] = Y[i].reshape((Y[i].shape[0], 1))
    
    Z = np.hstack((X[i], Y[i]))
    np.random.shuffle(Z)
    
    X[i] = Z[:, :Z.shape[1] - 1]
    Y[i] = Z[:, Z.shape[1] - 1]

72563it [00:10, 7115.87it/s]


In [9]:
xgbmodel = XGBRegressor(objective='reg:squarederror', n_estimators = 500)
# xgbmodel = XGBRegressor(objective='reg:squarederror', n_estimators = 100)
# xgbmodel = XGBRegressor(objective='reg:squarederror', n_estimators = 100, max_depth = 4)
xgbmodel.fit(X[0], Y[0])

/home/shashvat/anaconda3/lib/python3.7/site-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=500,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [10]:
print(mean_squared_error(xgbmodel.predict(X[1]), Y[1]))
print(np.exp(np.sqrt(mean_squared_error(xgbmodel.predict(X[1]), Y[1]))))

0.4495285951720879
1.955153949274481


In [11]:
pickle.dump(xgbmodel, open("models/xgbmodel500.model", "wb"))

In [12]:
for i in range(100):
    r = random.randrange(0, X[1].shape[1])
    x, y = X[1][[r]], Y[1][[r]]
    
    pred = xgbmodel.predict(x)[0] + x[:, -1]
    y = y + x[:, -1]
    pred = np.exp(pred) - np.exp(1)
    y = np.exp(y) - np.exp(1)
    
    print(y, pred, y / pred)

[9321.] [8622.10870113] [1.08105805]
[448.258] [460.1371862] [0.97418338]
[4501.] [3328.13336104] [1.35240975]
[557.8] [568.25878775] [0.98159503]
[56.194] [53.01011676] [1.0600618]
[6436.] [5791.18104453] [1.11134498]
[80.717] [77.74475952] [1.03823075]
[44.361] [50.8446663] [0.8724809]
[74.919] [84.67012048] [0.88483398]
[557.8] [590.47972212] [0.94465564]
[1000.12] [1290.9634818] [0.7747082]
[391.] [524.72286165] [0.74515526]
[166.757] [54.86535197] [3.03938632]
[191.589] [210.00127999] [0.91232301]
[2137.] [2608.63923567] [0.81920105]
[557.8] [568.25878775] [0.98159503]
[44.534] [42.8856276] [1.03843648]
[2099.] [2550.4722231] [0.82298485]
[58.158] [56.46901399] [1.02990996]
[122.441] [173.68288253] [0.70496872]
[609.5] [729.62440054] [0.83536132]
[433.7] [50.4635069] [8.59432938]
[421.] [409.73987662] [1.02748115]
[305.879] [272.70400624] [1.12165202]
[80.717] [76.95805561] [1.04884407]
[185.072] [222.30977795] [0.832496]
[56.194] [53.01011676] [1.0600618]
[74.919] [67.34939611] [

In [13]:
loaded_model = pickle.load(open("models/xgbmodel500.model", "rb"))
print(mean_squared_error(loaded_model.predict(X[1]), Y[1]))
print(np.exp(np.sqrt(mean_squared_error(loaded_model.predict(X[1]), Y[1]))))

0.4495285951720879
1.955153949274481
